In [17]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [11]:
q = Query.objects.get(pk=qid)

mdocs = Doc.objects.filter(
    query=q,
    wosarticle__cr__isnull=False
)

referred_citations = Citation.objects.filter(
    referent__isnull=False,
    referent__UT__in=mdocs.values_list('UT',flat=True)
)
print(referred_citations.count())
rcs = referred_citations.values_list('id',flat=True)

42476


In [8]:
cdos = CDO.objects.filter(
    doc__query=q,
    citation__in=rcs
)
print(cdos.count())

m = mdocs.count()
m_dict = dict(zip(
    list(mdocs.values_list('UT',flat=True)),
    list(range(m))
))


rev_m_dict = dict(zip(
    list(range(m)),
    list(mdocs.values_list('UT',flat=True))
))

print("ROWIDS")
row_ids = list(cdos.values_list('doc__UT',flat=True))
rows = np.array([m_dict[x] for x in row_ids])

print("colids")
col_ids = list(cdos.values_list('citation__referent__UT',flat=True))
cols = np.array([m_dict[x] for x in col_ids])

print("data")
data = np.array([1]*cdos.count())

print("matrix")
Scoo = coo_matrix((data, (rows,cols)),shape=(m,m))

275151
ROWIDS
colids
data
matrix


In [9]:
S = Scoo.tocsr()
S.shape

(164837, 164837)

In [14]:

save_sparse_csr("../networks/1457_citations",S)


In [11]:
S

<164837x164837 sparse matrix of type '<class 'numpy.int64'>'
	with 274481 stored elements in Compressed Sparse Row format>


## do again with Lutz' data

In [10]:
q = Query.objects.get(pk=qid)

mdocs = Doc.objects.filter(
    query=q
)

with open("../networks/1457_nodelabels_lutz.txt","w") as f:
    for i in mdocs.iterator():
        f.write("{}\t{}\n".format(i.title,i.UT))
        
with open("../cluster/1457_nodelabels_lutz.txt","w") as f:
    for i in mdocs.iterator():
        f.write("{}\t{}\n".format(i.title,i.UT))

m = mdocs.count()
m_dict = dict(zip(
    list(mdocs.values_list('UT',flat=True)),
    list(range(m))
))


rev_m_dict = dict(zip(
    list(range(m)),
    list(mdocs.values_list('UT',flat=True))
))


citers = []
cited = []
brows = 0

with open("../networks/jan_matched_data.csv","r") as jmd:
    c = csv.DictReader(jmd,delimiter=";")
    for row in c:
        try:
            c1 = m_dict[row["citing_ut"]]
            c2 = m_dict[row["cited_ut"]]
            citers.append(c1)
            cited.append(c2)
        except KeyError:
            brows+=1
print(len(citers))
print(brows)
            

528296
3514002


In [11]:
data = np.array([1]*len(citers))

Scoo = coo_matrix((data, (citers,cited)),shape=(m,m))

S = Scoo.tocsr()
S.shape

(172557, 172557)

In [12]:

save_sparse_csr("../networks/1457_citations_lutz",S)
save_sparse_csr("../cluster/1457_citations_lutz",S)

## again with bib coupling

In [30]:
django.db.connections.close_all()

q = Query.objects.get(pk=qid)

mdocs = Doc.objects.filter(
    query=q
)

global n_dict
n_dict = {}

def cit_ider(c):
    try:
        r = n_dict[c]
        #print(r)
    except:
        r = len(n_dict)
        n_dict[c] = r
    return(r)
        
citers = []
cited = []
brows = 0 

    
with open("../networks/jan_matched_data.csv","r") as jmd:
    c = csv.DictReader(jmd,delimiter=";")
    for row in c:
        try:
            c1 = m_dict[row["citing_ut"]]
            c2 = cit_ider(row["cited_ut"])
            citers.append(c1)
            cited.append(c2)
        except KeyError:
            brows+=1
            print(row)
            break
print(len(citers))
print(brows)
            

4042298
0


In [31]:
data = np.array([1]*len(citers))

n = len(n_dict)

Scoo = coo_matrix((data, (citers,cited)),shape=(m,n))

S = Scoo.tocsr()
S.shape

(172557, 1720102)

In [38]:
St = S.transpose()
Cmat = S*St
Cmat


multiply


In [39]:
Cmat

<172557x172557 sparse matrix of type '<class 'numpy.int64'>'
	with 484416277 stored elements in Compressed Sparse Row format>

In [49]:
s_3 = Cmat.multiply(Cmat > 2)
s_3

save_sparse_csr("../networks/1457_bc_lutz_3",s_3)
save_sparse_csr("../cluster/1457_bc_lutz_3",s_3)

<1x3194775 sparse matrix of type '<class 'numpy.int64'>'
	with 3194775 stored elements in Compressed Sparse Row format>